In [5]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)

#from pandas import DataFrame, read_csv
import pandas as pd

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn import metrics

import sys  
sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
from AdaBandFlt import *

%matplotlib tk

In [6]:
#load data
# file path of csv file
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [7]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

#xminnoise = int(np.round(11114*(fs/1000)))
#xmaxnoise = int(np.round(18511*(fs/1000)))

#noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

#xminspike = int(np.round(130826*(fs/1000)))
#xmaxspike = int(np.round(131699*(fs/1000)))

#burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

#begin_data = signal_filtered.iloc[:500000]

In [8]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered

In [9]:
fs = 25000

noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

"""plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')"""

"plt.figure()\nplt.plot(noise_levels)\nplt.grid(True)\nplt.xlabel('Time')\nplt.ylabel('Noise Amplitude [µV]')\nplt.title('Noise Levels')"

In [10]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.002, 
                           noise_window_size = 0.01,
                           threshold_factor = 3.5,
                           maxseparation = 0.0008)  
                
#spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']
spike_info

,indice_max,indice_min,indice_depass_positif,indice_depass_negatif,indice_1er_depass,indice_zero_central,i_max-i_min,Delta_amplitudes
0,53278,53286,53277,53285,53277,53281,-8,16.236913
1,53391,53375,53389,53373,53373,53386,16,18.401228
2,53922,53939,53910,53931,53910,53927,-17,44.085214
3,54115,54133,54106,54126,54106,54123,-18,61.369541
4,54314,54329,54302,54322,54302,54320,-15,59.717960
...,...,...,...,...,...,...,...,...
267,477408,477389,477407,477386,477386,477402,19,17.496226
268,477517,477528,477514,477526,477514,477522,-11,18.190074
269,477834,477815,477831,477813,477813,477827,19,19.565390
270,477905,477919,477897,477916,477897,477912,-14,29.808358


In [11]:
spike_data = record_spikes(signal, fs, spike_info,
                           'indice_1er_depass',
                           t_before = 0.001, 
                           t_after = 0.002)

(273, 76)


In [12]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                                           'indice_1er_depass',
                                           t_before = 0.001,
                                           t_after = 0.002)

In [13]:
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.grid(True)

In [14]:
print_spikes(spike_data,
             t_before_alignement = 0.001,
             first_spike = 1,
             last_spike = 50,
             fs = 25000)

In [15]:
pca_data = np.array(spike_data.iloc[:,1:].values).transpose()

In [16]:
## PLOT THE PCA

fig = plt.figure(4, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()
pca = PCA(n_components=5)
pca.fit(pca_data)
X = pca.transform(pca_data)

ax.scatter(X[:, 0], X[:, 1], X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

plt.show()

In [17]:
#dbscan.core_sample_indices_

#dbscan.components_

#dbscan.labels_

In [18]:
def print_clusters_3d(labels, X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    nb_clusters = max(labels) + 1

    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])

    for nb in range(nb_clusters):
        data = X[[x for x,y in b if y==nb],:]
        ax.scatter(data[:,0], data[:,1], data[:,2])
    data = X[[x for x,y in b if y==-1],:]
    ax.scatter(data[:,0], data[:,1], data[:,2], c='black')    
    
    ax.set_title('Nombre de cluster(s) :' + str(nb_clusters))
    plt.show()
    
"""print_clusters_3d(kmeans.labels_, X)"""

'print_clusters_3d(kmeans.labels_, X)'

In [19]:
def print_spikes_clusterized(spike_data,
                             labels,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000):
    
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 1
    else:
        nb_clusters_ = nb_clusters
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    #spike_data.iloc[:,first_spike:last_spike].plot()
    t_b = int(np.round(fs*(t_before_alignement)))
    y = (spike_data.iloc[:,0]-t_b)*1000/fs
        
    a = [i+1 for i in range(len(labels))]
    b = np.transpose([a,list(labels)])
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        data = spike_data.iloc[:,[x for x,y in b if y==nb]]
        m = len(data.values[0])
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster numero ' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        
    if (-1 in labels) == True:
        data = spike_data.iloc[:,[x for x,y in b if y==-1]]
        m = len(data.values[0])
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        
"""
print_spikes_clusterized(spike_data,
                             kmeans.labels_,
                             t_before_alignement = 0.0015,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000)
"""

'\nprint_spikes_clusterized(spike_data,\n                             kmeans.labels_,\n                             t_before_alignement = 0.0015,\n                             nb_spike = 20,\n                             y_lim_min = -50,\n                             y_lim_max = 60,\n                             fs = 25000)\n'

In [21]:
#dbscan = DBSCAN(eps=3, min_samples=2).fit(X)

## pour new_spike_data
##25//8 --> 2//83
##12//4 --> 7//194

## pour old_spike_data
##13//8 --> 8//4621

dbscan = DBSCAN(eps=12, min_samples=2, metric='euclidean').fit(X)

core_samples_mask = np.zeros_like(dbscan.labels_, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True
labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

Estimated number of clusters: 10
Estimated number of noise points: 252


In [322]:
print_clusters_3d(dbscan.labels_, X)

In [323]:
print_spikes_clusterized(spike_data,
                             dbscan.labels_,
                             t_before_alignement = 0.0015,
                             nb_spike = 20,
                             y_lim_min = -40,
                             y_lim_max = 50,
                             fs = 25000)